## AwareNet-CLIP evaluation on CIFAR10

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import pathlib
import numpy as np
import os
import clip
from awareness import awareness
import psutil
import nvidia_smi
from torchvision.datasets import CIFAR10

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

process = psutil.Process(os.getpid())
init_res = process.memory_info().rss
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
init_gpu = info.used
nvidia_smi.nvmlShutdown()

Device: cuda


In [10]:
BATCH_SIZE = 1024
DYNAMIC_RAY = True

CHK_PATH = 'checkpoints/awarenet-clip_cifar10/awarenet-clip_cifar10_16_0.8904_10745'
CLIP_PATH = f'{CHK_PATH}/clip.pt'
AWAR_PATH = f'{CHK_PATH}/awarenet.pt'

In [12]:
preprocess = transforms.Compose([
    transforms.Resize(size=224, interpolation=PIL.Image.BICUBIC),
    transforms.CenterCrop(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), 
        std=(0.26862954, 0.26130258, 0.27577711)),
])

clip_model = torch.load(CLIP_PATH)
clip_model.eval().to(device)

awareness_model = torch.load(AWAR_PATH, map_location=torch.device(device))
awareness_model.to(device)

NameError: name 'PIL' is not defined

In [8]:
cifar10_train = CIFAR10(os.path.expanduser("~/.cache"), train=True, transform=preprocess, download=True)
cifar10_test = CIFAR10(os.path.expanduser("~/.cache"), train=False, transform=preprocess, download=True)

test_loader = DataLoader(
    cifar10_test,
    batch_size=BATCH_SIZE,
    shuffle=True
)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)

with torch.no_grad(): 

    test_correct_preds_batches = []
    test_count = 0

    clip_model.eval()
    awarenet_model.eval()
    
    references = awarenet_model.awareness.ref_insts
    
    n_ref_insts = len(references)
    
    inf_times = []

    for i, (images, labels) in enumerate(test_loader):

        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        starter.record()

        features = clip_model.encode_image(images).float()
        preds = awarenet_model.forward(torch.unsqueeze(features,1))
        
        ender.record()
        torch.cuda.synchronize()
        inf_times.append(starter.elapsed_time(ender))

        test_correct_preds_batch = utils.correct_preds_numpy(np.array(preds), labels.cpu().numpy())
        test_correct_preds_batches.append(test_correct_preds_batch)
        test_count = test_count+len(images)

        test_batch_accuracy = round(np.sum(test_correct_preds_batch)/preds.size(0), 4)

        run_res = process.memory_info().rss
            
        nvidia_smi.nvmlInit()
        handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
        info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
        run_gpu = info.used
        nvidia_smi.nvmlShutdown()
                    
    test_accuracy = round(np.sum(test_correct_preds_batches)/test_count, 4)

    RAM_usg = round((run_res-init_res)*1e-09,2)
    GPU_usg = round((run_gpu-init_gpu)*1e-09,2)

    inf_times_np = np.array(inf_times)
    mean_syn = round(np.sum(inf_times_np)/inf_times_np.shape[0],2)
    std_syn = round(np.std(inf_times_np),2)
    
    print(f'Reference instances (N): {n_ref_insts}, Test accuracy: {test_accuracy}')
    print(f'RAM usage: {RAM_usg} GB')
    print(f'GPU usage: {GPU_usg} GB')
    print(f'Inference time: {mean_syn} ± {std_syn} ms')

Reference instances (N): 10745, Test accuracy: 0.8902
